# Install Libraries and Import Deep Learning model

In [1]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.6 MB/s eta 0:00:00


In [7]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

In [8]:
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")

emotion = pipeline('text-classification', 
                    model='arpanghoshal/EmoRoBERTa')


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [9]:
emotion_labels = emotion("Hi! Alexander. Nice to connect with you. Just change the value of K to get top labels! :)")
print(emotion_labels)

[{'label': 'joy', 'score': 0.6877943873405457}]


# Define Functions to process email subject lines

In [10]:
emotions_list = ['admiration', 'amusement', 'anger', 'annoyance', 'approval',
'caring', 'confusion', 'curiosity', 'desire',
'disappointment',
'disapproval',
'disgust',
'embarrassment',
'excitement',
'fear',
'gratitude',
'grief',
'joy',
'love',
'nervousness',
'optimism',
'pride',
'realization',
'relief',
'remorse',
'sadness',
'surprise',
'neutral']

In [14]:

def build_classification_results_table(list_of_text_inputs):
  ''' 
  function to get classification results for each row for all 28 emotions and turn it into a data frame
  '''

  scores_total = pd.DataFrame(columns=emotions_list)

  for text_input in list_of_text_inputs:
    emotion_object = emotion(text_input, top_k = 28)
    list_o_emotions = []
    list_o_scores = []
    
    for i in range(len(emotion_object)):
      list_o_emotions.append(emotion_object[i]['label'])
      list_o_scores.append(emotion_object[i]['score'])
    
    scores_WIP = pd.DataFrame(zip(list_o_emotions, list_o_scores)).T 
    scores_WIP = scores_WIP.rename(columns=scores_WIP.iloc[0]).loc[1:]
    scores_WIP = scores_WIP.reindex(emotions_list, axis='columns')
    
    scores_total = scores_total.append(scores_WIP, ignore_index=True)
    #scores_total = pd.concat([scores_total, scores_WIP])
    #scores_total.concat(scores_WIP)


  return scores_total

In [ ]:
scores_total = pd.DataFrame(columns=emotions_list)

# Load Data 

In [15]:
from google.colab import files
 
 
uploaded = files.upload()

Saving Copy of q4 2022 data raw.csv to Copy of q4 2022 data raw.csv


In [16]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['Copy of q4 2022 data raw.csv']))
df.head()

,send_dt,send_time,send_number,campaign,email_name,category,text,emails_sent,emails_delivered,undeliverable,...,Audience,send_group,Open_Rate_nw,Click_Rate_nw,Donation_Rate_nw,revenue_1k_new,month,polarity_score,subjectivity_score,processed_text
0,2022-08-26,14:49:44,1,Unknown,ML Insights Survey Monthly 2022 August - Reminder,Unknown,Insight Panel: Our team answers your questions,468,468,0,...,Other,ML Insights Surv,0.570513,0.070513,0.0,0.0,8,0.0,0.0,insight panel team answer question
1,2022-08-26,20:06:50,1,Monkeypox,2022-08-25-Monkeypox-0-12-Month-Donors,Cultivation,Here are th,131132,130411,721,...,First Levels,2022-08-25-Monke,0.322957,0.003981,0.0,0.0,8,0.0,0.0,th
2,2022-08-26,20:06:53,1,Monkeypox,2022-08-25-Monkeypox-Non-Donors,Cultivation,Here are th,224396,223572,824,...,Non Donors,2022-08-25-Monke,0.279720,0.002130,0.0,0.0,8,0.0,0.0,th
3,2022-08-26,20:07:59,1,Monkeypox,2022-08-25-Monkeypox-Field-Partner,Cultivation,Here are th,76001,75790,211,...,Field Partners,2022-08-25-Monke,0.397429,0.004934,0.0,0.0,8,0.0,0.0,th
4,2022-08-26,20:07:41,1,Monkeypox,2022-08-25-Monkeypox-Partner,Cultivation,Here are th,18226,18182,44,...,Partners,2022-08-25-Monke,0.403105,0.004334,0.0,0.0,8,0.0,0.0,th


In [18]:
df.at[0,'text']

'Insight Panel: Our team answers your questions'

In [17]:
emotion(df.at[0,'text'])

[{'label': 'neutral', 'score': 0.9787247776985168}]

# Apply emotion scoring using hugging face model

In [20]:
twerp = build_classification_results_table(list(df.text[0:10]))

twerp.head(25)


,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,0.001167,0.000987,0.000026,0.000121,0.006524,0.000135,0.001798,0.00155,0.000055,0.000077,...,0.000023,0.00002,0.00033,0.000054,0.000193,0.000344,0.000067,0.00003,0.000071,0.978725
1,0.001001,0.001408,0.00008,0.000185,0.000189,0.00003,0.000305,0.000685,0.000048,0.000077,...,0.000031,0.000008,0.00003,0.000012,0.000028,0.000007,0.000018,0.000054,0.000084,0.995067
2,0.001001,0.001408,0.00008,0.000185,0.000189,0.00003,0.000305,0.000685,0.000048,0.000077,...,0.000031,0.000008,0.00003,0.000012,0.000028,0.000007,0.000018,0.000054,0.000084,0.995067
3,0.001001,0.001408,0.00008,0.000185,0.000189,0.00003,0.000305,0.000685,0.000048,0.000077,...,0.000031,0.000008,0.00003,0.000012,0.000028,0.000007,0.000018,0.000054,0.000084,0.995067
4,0.001001,0.001408,0.00008,0.000185,0.000189,0.00003,0.000305,0.000685,0.000048,0.000077,...,0.000031,0.000008,0.00003,0.000012,0.000028,0.000007,0.000018,0.000054,0.000084,0.995067
5,0.001001,0.001408,0.00008,0.000185,0.000189,0.00003,0.000305,0.000685,0.000048,0.000077,...,0.000031,0.000008,0.00003,0.000012,0.000028,0.000007,0.000018,0.000054,0.000084,0.995067
6,0.000108,0.000138,0.000031,0.000121,0.00635,0.00812,0.000514,0.001562,0.000934,0.000139,...,0.000045,0.000063,0.001629,0.000003,0.000052,0.000081,0.000243,0.000551,0.00002,0.978542
7,0.000108,0.000138,0.000031,0.000121,0.00635,0.00812,0.000514,0.001562,0.000934,0.000139,...,0.000045,0.000063,0.001629,0.000003,0.000052,0.000081,0.000243,0.000551,0.00002,0.978542
8,0.000108,0.000138,0.000031,0.000121,0.00635,0.00812,0.000514,0.001562,0.000934,0.000139,...,0.000045,0.000063,0.001629,0.000003,0.000052,0.000081,0.000243,0.000551,0.00002,0.978542
9,0.000108,0.000138,0.000031,0.000121,0.00635,0.00812,0.000514,0.001562,0.000934,0.000139,...,0.000045,0.000063,0.001629,0.000003,0.000052,0.000081,0.000243,0.000551,0.00002,0.978542


In [21]:
twerp = build_classification_results_table(list(df.text[0:50]))

twerp.to_csv('Q4 2022 emotion outputs n_50 v1.csv', index=False)

In [22]:
twerp = build_classification_results_table(list(df.text))

twerp.to_csv('Q4 2022 emotion outputs full v1.csv', index=False)

In [23]:
twerp.shape

(559, 28)

# Appendix_old code snippets

### testing the function this needs to go into the appropriate place later

In [ ]:
def get_emotion_label(text):
  emotion_object = emotion(text)
  emotion_label = emotion_object[0]['label']
  score = emotion_object[0]['score']
  return emotion_label, score

#def get_emotion_score(text):
  #emotion_label = emotion(text)[0]['label']
  #score = emotion(text)[0]['score']
  #return score #, emotion_label

In [ ]:
emotion_object = emotion(df.text[3], top_k = 28)
emotion_object
#len(emotion_object)

In [ ]:
emotion_object = emotion(df.text[3], top_k = 28)
list_o_emotions = []
list_o_scores = []

for i in range(len(emotion_object)):
      list_o_emotions.append(emotion_object[i]['label'])
      list_o_scores.append(emotion_object[i]['score'])

scores_WIP = pd.DataFrame(zip(list_o_emotions, list_o_scores)).T 
scores_WIP = scores_WIP.rename(columns=scores_WIP.iloc[0]).loc[1:]
scores_WIP = scores_WIP.reindex(emotions_list, axis='columns')
scores_WIP

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
1,0.000384,0.000431,0.000582,0.012102,0.001591,0.000039,0.000692,0.000014,0.000079,0.00817,...,0.000114,0.000199,0.000435,0.000112,0.035394,0.000057,0.000143,0.000768,0.000088,0.880445


In [ ]:
scores_total = scores_total.append(scores_WIP, ignore_index=True)
scores_total

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,0.000384,0.000431,0.000582,0.012102,0.001591,0.000039,0.000692,0.000014,0.000079,0.00817,...,0.000114,0.000199,0.000435,0.000112,0.035394,0.000057,0.000143,0.000768,0.000088,0.880445


In [ ]:
scores_total is scores_WIP

False

### Factor Analysis and Clustering


In [ ]:
new_df.sum()

admiration                                                        1
amusement                                                         0
anger                                                             0
annoyance                                                         0
approval                                                        144
caring                                                          188
confusion                                                        13
curiosity                                                       296
desire                                                           66
disappointment                                                   18
disapproval                                                       5
disgust                                                           7
embarrassment                                                     0
excitement                                                      104
fear                                            

In [ ]:
new_df.to_csv('export with hf classification.csv', index=False)

In [ ]:
df.to_csv('original df.csv', index=False)

In [ ]:
df.text[0:10].apply(get_emotion_label)

NameError: ignored

In [ ]:
new_df = pd.DataFrame(columns=emotions_list)

In [ ]:
new_df[['emotion_label', 'emotion_score']] = pd.DataFrame(df.text.apply(get_emotion_label).to_list(), index=df.index)

In [ ]:
new_df.head()

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,emotion_label,emotion_score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,approval,0.280771
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,0.969983
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,0.969983
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sadness,0.542841
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sadness,0.542841


## V2: getting scores for all 28 possible emotion readings


In [ ]:
emotion_labels = emotion("TensorFlow is really difficult to work with which is why Google created Collab notebooks", top_k=28)

#emotion_labels = emotion("Hi! Alexander. Nice to connect with you. Just change the value of K to get top labels! :) ,", top_k=28)
#print(emotion_labels)

In [ ]:
emotion_labels

In [ ]:
emotion_labels[0]['score']

0.7129493951797485

In [ ]:
list_o_emotions = []
list_o_scores = []

for i in range(len(emotion_labels)):
  list_o_emotions.append(emotion_labels[i]['label'])
  list_o_scores.append(emotion_labels[i]['score'])

In [ ]:
#list_o_emotions
#list_o_scores


In [ ]:
scores_WIP = pd.DataFrame(zip(list_o_emotions, list_o_scores)).T #columns=['emotions','scores']

In [ ]:
scores_WIP

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,joy,approval,caring,admiration,love,relief,optimism,neutral,excitement,gratitude,...,confusion,disappointment,grief,disapproval,fear,remorse,anger,nervousness,sadness,embarrassment
1,0.712949,0.241367,0.011121,0.010838,0.005513,0.005508,0.003131,0.002087,0.001837,0.001815,...,0.000118,0.000089,0.000069,0.000064,0.000046,0.000043,0.00004,0.000036,0.000035,0.000016


In [ ]:
scores_WIP = scores_WIP.rename(columns=scores_WIP.iloc[0]).loc[1:]

In [ ]:
scores_WIP

,joy,approval,caring,admiration,love,relief,optimism,neutral,excitement,gratitude,...,confusion,disappointment,grief,disapproval,fear,remorse,anger,nervousness,sadness,embarrassment
1,0.712949,0.241367,0.011121,0.010838,0.005513,0.005508,0.003131,0.002087,0.001837,0.001815,...,0.000118,0.000089,0.000069,0.000064,0.000046,0.000043,0.00004,0.000036,0.000035,0.000016


In [ ]:
scores_WIP = scores_WIP.reindex(emotions_list, axis='columns')

In [ ]:
scores_WIP

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
1,0.010838,0.000293,0.00004,0.000139,0.241367,0.011121,0.000118,0.000657,0.000218,0.000089,...,0.005513,0.000036,0.003131,0.000231,0.001466,0.005508,0.000043,0.000035,0.000136,0.002087


In [ ]:
new_df = pd.DataFrame(columns=emotions_list)

In [ ]:
for row in new_df.index:
  for col in new_df:
    new_df.at[row,emotion_classification_result] = scores_WIP.at[emotion_classification_result,]

## Putting Emotion scores  into a DF

In [ ]:
for row in new_df.index:
  top_emotion = new_df.at[row,'emotion_label']
  new_df.at[row,top_emotion] = 1

In [ ]:
new_df.head()

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,emotion_label,emotion_score
0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,approval,0.280771
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,neutral,0.969983
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,neutral,0.969983
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,sadness,0.542841
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,sadness,0.542841


In [ ]:
new_df.fillna(0, inplace=True)

In [ ]:
new_df.head(50)

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,emotion_label,emotion_score
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,approval,0.280771
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,neutral,0.969983
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,neutral,0.969983
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,sadness,0.542841
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,sadness,0.542841
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,excitement,0.991233
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,excitement,0.991233
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,excitement,0.991233
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,sadness,0.909134
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,sadness,0.909134


## Import Data and Run Function

In [ ]:
df.text[0:10]

0    MSF's Ukraine response and how you can make a ...
1                           You make our work possible
2                           You make our work possible
3               DEADLINE: We?re just short of our goal
4               DEADLINE: We?re just short of our goal
5               FWD: We?re so close to the finish line
6               FWD: We?re so close to the finish line
7               FWD: We?re so close to the finish line
8              Fires leave Moria refugee camp in ashes
9              Fires leave Moria refugee camp in ashes
Name: text, dtype: object

In [ ]:
list(df.text[0:10])

["MSF's Ukraine response and how you can make a greater impact",
 'You make our work possible',
 'You make our work possible',
 'DEADLINE: We?re just short of our goal',
 'DEADLINE: We?re just short of our goal',
 'FWD: We?re so close to the finish line',
 'FWD: We?re so close to the finish line',
 'FWD: We?re so close to the finish line',
 'Fires leave Moria refugee camp in ashes',
 'Fires leave Moria refugee camp in ashes']

In [ ]:
twerp.shape

(89, 28)